# Orientation/Direction Tuning
- Compute tuning curves.
- Determine orientation/direction selectivity of cells using permutation test (Ecker et al., 2014, Baden et al., 2016).

In [9]:
import os
import numpy as np
import pandas as pd

import datajoint as dj

from lib.data import *
from lib.tuning import *
from lib.calcium import *
from lib.plot import *
from lib.utils import *

import matplotlib.pyplot as plt

## Load data tables

In [10]:
dataFolder = "/srv/public/datasets/VIS-L23/"
functionalDataFolder = os.path.join(dataFolder, "data_Microns_L23", "211019_vignette_functional_analysis_data", "function_data_tables")

Stimulus = load_dict(os.path.join(functionalDataFolder,"Stimulus.pkl"))
EASETrace = load_dict(os.path.join(functionalDataFolder,"EASETrace.pkl"))
EASETuning = load_dict(os.path.join(functionalDataFolder,"EASETuning.pkl"))

print(EASETrace.keys())

dict_keys(['scan_id', 'segmentation', 'segment_id', 'manual_id', 'trace_raw', 'trace', 'spike'])


## Pyramidal cells with functional data

In [11]:
seg_id_list = EASETuning["segment_id"]
scan_list = EASETuning["scan_id"]
n_cells = seg_id_list.shape[0]

In [12]:
with open(os.path.join(dataFolder, "functional_data", "ids_scans.csv"), "w") as f:
    f.write("neuron_id,scan_id\n")
    for idx in range(0,len(seg_id_list)):
        segId = seg_id_list[idx]
        scanId = scan_list[idx]
        f.write("{},{}\n".format(segId, scanId))

In [14]:

def writeIds(filename, ids):
    with open(filename, "w") as f:
        for neuronId in ids:
            f.write("{}\n".format(neuronId))

def processAndSaveTraceData(scanId, traceData, traceType, scanIds):
    corrMatrix = np.corrcoef(traceData)
    filenameMatrix = os.path.join(dataFolder, "functional_data", "correlation_{}_{}".format(traceType, scanId))
    np.savetxt(filenameMatrix, corrMatrix)

def writeIdsForScan(scanId, ids):
    filenameIds = os.path.join(dataFolder, "functional_data", "ids_scan_{}".format(scanId))
    writeIds(filenameIds, ids) 

for traceType in ["trace_raw", "trace", "spike"]:
    lastScanId = -1
    traceData = None
    scanNeuronIds = []
    for idx in range(0,len(seg_id_list)):
        segId = seg_id_list[idx]
        scanId = scan_list[idx]    
        if(scanId != lastScanId):
            if(traceData is not None):
                processAndSaveTraceData(lastScanId, traceData, traceType, scanNeuronIds)         
                writeIdsForScan(scanId, scanNeuronIds)
            traceData = []
            scanNeuronIds = []
            lastScanId = scanId
        scanNeuronIds.append(segId)
        traceData.append(get_trace(EASETrace, segId, scanId, traceType))
        
    processAndSaveTraceData(lastScanId, traceData, traceType, scanNeuronIds)
    writeIdsForScan(scanId, scanNeuronIds)

In [15]:
# save tuning curves to file

with open(os.path.join(dataFolder, "functional_data", "tuning_curves.csv"), "w") as f:
    f.write("neuron_id")
    for k in range(1, 17):
        f.write(",D{}".format(k))
    f.write("\n")
    
    for i in range(n_cells):    
        seg_id = seg_id_list[i]
        f.write("{}".format(seg_id))
        
        scan_id = int(scan_list[seg_id_list==seg_id])

        trace = get_trace(EASETrace, seg_id, scan_id, "trace_raw")
        stimlab = get_stim_label(Stimulus, scan_id)

        response_array = get_peakamp_tdarray(trace, stimlab)
        tune = tuning_curve(response_array)
        
        for k in range(0, 16):
            f.write(",{:.6f}".format(tune[k]))
        f.write("\n")
        


## Determine orientation/direction tuned cells with permutation test
- $p<0.01$ are significantly tuned cells.

In [ ]:
T = 10000 # Number of iterations for permutation test

dsi_list = np.zeros(n_cells)
osi_list = np.zeros(n_cells)
dsi_pvalue = np.zeros(n_cells)
osi_pvalue = np.zeros(n_cells)
for i in range(n_cells):
    
    seg_id = seg_id_list[i]
    scan_id = int(scan_list[seg_id_list==seg_id])

    trace = get_trace(EASETrace, seg_id, scan_id, "trace_raw")
    stimlab = get_stim_label(Stimulus, scan_id)
    
    response_array = get_peakamp_tdarray(trace, stimlab)
    tune = tuning_curve(response_array)
    
    print(tune)
    
    dsi_list[i] = dsi(tune)
    osi_list[i] = osi(tune)
    dsi_shuf, dsi_pvalue[i] = permutation_test(response_array, T, "dsi")
    osi_shuf, osi_pvalue[i] = permutation_test(response_array, T, "osi")
    
    if (i+1)%10==0:
        print("{} / {} complete.".format(i+1, n_cells))

In [ ]:
# Same format as data/cell_functional_info.csv
data = {"segment_id": seg_id_list,
     "scan": scan_list,
     "osi": osi_list, 
     "osi_p": osi_pvalue,
     "dsi": dsi_list,
     "dsi_p": dsi_pvalue}

df = pd.DataFrame(data=data)
df